# Simpsons Haiku Demo

For anyone who knows me, The Simpsons has had an undue impact on my life, being formative through my early years and enduring to the present moment. Whether that contribution be in curating a sense of humour deeply anchored in an appreciation for the surreal, a continued love of animation, or exposure to the vaults of obscure Americana (thank you, John Swartwzelder).

In [1]:
from haiku import *

In [2]:
haiku = SimpsonsHaiku()

C:\Users\i_can\OneDrive\Documents\simpsons-haiku\haiku.py:31: DtypeWarning: Columns (4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  script_lines = pd.read_csv(self.file_path, on_bad_lines=on_bad_lines,
C:\Users\i_can\OneDrive\Documents\simpsons-haiku\haiku.py:47: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  script_lines['spoken_words_split'] = script_lines['spoken_words'].str.replace('?', '.').str.replace('!', '.').str.replace('/', '.').str.split('.')  # split on '.!?/', but could extend to ':;,'
100%|███████████████████████████████████████████████████████████████████████| 401253/401253 [00:07<00:00, 55113.60it/s]


In [3]:
tqdm.pandas()
script = haiku.script#.sample(100, random_state=0)

In [4]:
script.spoken_words.iloc[0]

'Ooo, careful, Homer.'

In [5]:
script[script.normalized_text.progress_apply(lambda x: 'diddly' in x)].iloc[0].values

100%|██████████████████████████████████████████████████████████████████████| 225200/225200 [00:00<00:00, 798566.52it/s]


array([25369, 86, 69,
       'Devil: Ahem! I hold here a contract between myself and one Homer Simpson, pledging me his soul for a donut. Which I delivered! And it was scrum-diddly-umptious!',
       378000, True, 346.0, 25.0, 'Devil', 'Simpson Living Room',
       'Ahem! I hold here a contract between myself and one Homer Simpson, pledging me his soul for a donut. Which I delivered! And it was scrum-diddly-umptious!',
       'ahem i hold here a contract between myself and one homer simpson pledging me his soul for a donut which i delivered and it was scrum-diddly-umptious',
       26.0, 'Ahem', 1, 2], dtype=object)

In [6]:
corpus = script['spoken_words'].str.cat(sep=' ')
for char in [",", ".", "?", "!", ":", "\\", "\""]:#self.strip_list:
    corpus = corpus.replace(char, '')
corpus_list = corpus.lower().replace('-', ' ').replace('/', ' ').split(' ')

corpus_df = pd.DataFrame({'word' : corpus_list})

simpsons_count = corpus_df.value_counts().reset_index(name='counts')
simpsons_count

,word,counts
0,the,94788
1,i,80963
2,you,79947
3,a,71775
4,,71515
...,...,...
42120,hidee,1
42121,hiedilly,1
42122,stormed,1
42123,hierarchy,1


In [7]:
import syllables 
# Words in need of labelling
# d = cmudict.dict()
# simpsons_count[simpsons_count.word.apply(lambda x: x not in d)].set_index('word')['counts'].to_dict()#json('new_words.json')

In [8]:
df = pd.read_json('simpson_lect.json', orient='index').reset_index().rename({'index':'word', 0:'n_syllable'}, axis=1)
df['syllables_estimate'] = df.word.apply(syllables.estimate)
df['syllapy_estimate'] = df.word.apply(syllapy.count)

df['syllables_error'] = abs(df.word.apply(syllables.estimate) - df['n_syllable'])
df['syllapy_error'] = abs(df.word.apply(syllapy.count) - df['n_syllable'])

In [9]:
# Comparing syllables and syllapy performance on labelled syllable set
df.describe().iloc[1:3, :]

,n_syllable,syllables_estimate,syllapy_estimate,syllables_error,syllapy_error
mean,1.866197,1.734742,1.617371,0.248826,0.295775
std,0.807863,0.780687,0.821235,0.573174,0.694063


In [10]:
haiku_df = haiku.generate_haiku_df(save=True)

100%|█████████████████████████████████████████████████████████████████████████| 30373/30373 [00:02<00:00, 13211.19it/s]


In [12]:
haiku_df.sample().values

array([[list([96058, 96058, 96058]), list([191, 191, 191]),
        list(["10-Year-Old Homer: Absolutely. Or my name isn't... whatever I told you it was.", "10-Year-Old Homer: Absolutely. Or my name isn't... whatever I told you it was.", "10-Year-Old Homer: Absolutely. Or my name isn't... whatever I told you it was."]),
        list([795000, 795000, 795000]), list([True, True, True]),
        list([4169.0, 4169.0, 4169.0]), list([2795.0, 2795.0, 2795.0]),
        list(['10-Year-Old Homer', '10-Year-Old Homer', '10-Year-Old Homer']),
        list(['PROMONTORY', 'PROMONTORY', 'PROMONTORY']),
        list(["Absolutely. Or my name isn't... whatever I told you it was.", "Absolutely. Or my name isn't... whatever I told you it was.", "Absolutely. Or my name isn't... whatever I told you it was."]),
        list(['absolutely or my name isnt whatever i told you it was', 'absolutely or my name isnt whatever i told you it was', 'absolutely or my name isnt whatever i told you it was']),
        lis

In [13]:
# One-liner haikus
haiku_df[haiku_df.n_syllables.apply(lambda x: x == [17])].head()

,id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,spoken_words_split,number_in_line,n_syllables
episode_id,,,,,,,,,,,,,,,
1,[34],[33],"[Bart Simpson: Oh, please. (TO LISA) There's o...",[226000],[True],[8],[5.0],[Bart Simpson],[Simpson Home],"[Oh, please. There's only one fat guy that bri...",[oh please theres only one fat guy that brings...,[16.0],There's only one fat guy that brings us prese...,[2],[17]
1,[42],[41],"[Homer Simpson: Yeah. If you want one, you'll ...",[258000],[True],[2],[5.0],[Homer Simpson],[Simpson Home],"[Yeah. If you want one, you'll have to pay for...",[yeah if you want one youll have to pay for it...,[16.0],"If you want one, you'll have to pay for it ou...",[2],[17]
2,[462],[84],[Seymour Skinner: By itself something like thi...,[443000],[True],[3],[28.0],[Seymour Skinner],[Skinner's Office],[By itself something like this might not call ...,[by itself something like this might not call ...,[40.0],By itself something like this might not call f...,[1],[17]
2,[522],[144],[Ms. Melon: Let me say right at the start that...,[683000],[True],[44],[3.0],[Ms. Melon],[Springfield Elementary School],[Let me say right at the start that we have on...,[let me say right at the start that we have on...,[38.0],Let me say right at the start that we have one...,[1],[17]
2,[628],[250],"[Dr. J. Loren Pyror: Now, Bart, we want to emp...",[1131000],[True],[43],[32.0],[Dr. J. Loren Pyror],[OLD SCHOOL],"[Now, Bart, we want to emphasize that nobody's...",[now bart we want to emphasize that nobodys an...,[54.0],"Now, Bart, we want to emphasize that nobody's ...",[1],[17]


In [14]:
# Max number of lines of dialogue in a 17-syllable sequence? It's 16 (3 men and a comic book)
haiku_df['n_lines'] = haiku_df.number.apply(len)
haiku_df[haiku_df.n_lines == haiku_df.n_lines.max()]

,id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,spoken_words_split,number_in_line,n_syllables,n_lines
episode_id,,,,,,,,,,,,,,,,
195,"[54908, 54908, 54908, 54909, 54909, 54909, 549...","[135, 135, 135, 136, 136, 136, 136, 136, 136, ...","[Bart Simpson: Yes! Heh, heh! / Ha! Ha! / Woo ...","[629000, 629000, 629000, 639000, 639000, 63900...","[True, True, True, True, True, True, True, Tru...","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[Bart Simpson, Bart Simpson, Bart Simpson, Bar...","[Simpson Home, Simpson Home, Simpson Home, Sim...","[Yes! Heh, heh! / Ha! Ha! / Woo hoo hoo heh he...",[yes heh heh ha ha woo hoo hoo heh heh woo oh ...,"[13.0, 13.0, 13.0, 11.0, 11.0, 11.0, 11.0, 11....",Woo Oh Oh Yeah Ha Ha Ha Ha Ha Ha Ha ...,"[9, 11, 12, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 1,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1]",16
195,"[54908, 54908, 54909, 54909, 54909, 54909, 549...","[135, 135, 136, 136, 136, 136, 136, 136, 136, ...","[Bart Simpson: Yes! Heh, heh! / Ha! Ha! / Woo ...","[629000, 629000, 639000, 639000, 639000, 63900...","[True, True, True, True, True, True, True, Tru...","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[Bart Simpson, Bart Simpson, Bart Simpson, Bar...","[Simpson Home, Simpson Home, Simpson Home, Sim...","[Yes! Heh, heh! / Ha! Ha! / Woo hoo hoo heh he...",[yes heh heh ha ha woo hoo hoo heh heh woo oh ...,"[13.0, 13.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11....",Oh Oh Yeah Ha Ha Ha Ha Ha Ha Ha Ha ...,"[11, 12, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 1, 2,...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1]",16
195,"[54908, 54909, 54909, 54909, 54909, 54909, 549...","[135, 136, 136, 136, 136, 136, 136, 136, 136, ...","[Bart Simpson: Yes! Heh, heh! / Ha! Ha! / Woo ...","[629000, 639000, 639000, 639000, 639000, 63900...","[True, True, True, True, True, True, True, Tru...","[8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, 8.0, ...","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[Bart Simpson, Bart Simpson, Bart Simpson, Bar...","[Simpson Home, Simpson Home, Simpson Home, Sim...","[Yes! Heh, heh! / Ha! Ha! / Woo hoo hoo heh he...",[yes heh heh ha ha woo hoo hoo heh heh woo oh ...,"[13.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11.0, 11....",Oh Yeah Ha Ha Ha Ha Ha Ha Ha Ha All ...,"[12, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 1, 2, 3, ...","[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 1, 1, 1, 1, 1]",16


In [15]:
# Max number of unique characters in a 17-syllable sequence? It's Homer 3D, with 9 characters. Did anyone see the movie Tron?
haiku_df['n_characters'] = haiku_df.character_id.apply(lambda x: len(set(x)))
haiku_df[haiku_df.n_characters == haiku_df.n_characters.max()]

,id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,spoken_words_split,number_in_line,n_syllables,n_lines,n_characters
episode_id,,,,,,,,,,,,,,,,,
134,"[38660, 38661, 38662, 38663, 38664, 38665, 386...","[223, 224, 225, 226, 227, 228, 229, 230, 231, ...","[Marge Simpson: No., Bart Simpson: No., Patty ...","[1105000, 1106000, 1106000, 1106000, 1107000, ...","[True, True, True, True, True, True, True, Tru...","[1.0, 8.0, 10.0, 71.0, 11.0, 22.0, 442.0, 140....","[5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, 5.0, ...","[Marge Simpson, Bart Simpson, Patty Bouvier, C...","[Simpson Home, Simpson Home, Simpson Home, Sim...","[No., No., No., No., No., No., No., No., Yes. ...","[no, no, no, no, no, no, no, no, yes i mean no...","[1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 5.0, ...",No No No No No No No No Yes I mean no No Wel...,"[1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 3, 1]","[1, 1, 1, 1, 1, 1, 1, 1, 1, 3, 1, 4]",12,9


In [16]:
# How about locations?
haiku_df['n_locations'] = haiku_df.location_id.apply(lambda x: len(set(x)))
haiku_df[haiku_df.n_locations == haiku_df.n_locations.max()].spoken_words_split.values

array([" Almost sensuous Oh, baby Damn boxes He's trapped Where's our backup",
       'I mean Okay, they bought it Have you got Lou in your car Roger that'],
      dtype=object)

In [17]:
# Taking a look at the longest lines
long_script = script[script.n_syllables > 17]
long_script.sort_values('n_syllables', ascending=False)

,id,episode_id,number,raw_text,timestamp_in_ms,speaking_line,character_id,location_id,raw_character_text,raw_location_text,spoken_words,normalized_text,word_count,spoken_words_split,number_in_line,n_syllables
74110,84048,291,160,"Homer Simpson: You know, I've had a lot of job...",615000,True,2.0,5.0,Homer Simpson,Simpson Home,"You know, I've had a lot of jobs: boxer, masco...",you know ive had a lot of jobs boxer mascot as...,69.0,"You know, I've had a lot of jobs: boxer, masco...",1,117
146969,157505,565,160,"Homer Simpson: Chubby, chunky, blobbo, slobbo,...",750000,True,2,15.0,Homer Simpson,Moe's Tavern,"Chubby, chunky, blobbo, slobbo, Fat Bastard, M...",chubby chunky blobbo slobbo fat bastard michel...,84.0,", Papa Grandé, Augustus Gloop, Beached Whale, ...",2,99
153582,4860,16,272,"V.O.: And the Raven, never flitting, still is ...",1252000,True,265,224.0,V.O.,CHAMBER,"And the Raven, never flitting, still is sittin...",and the raven never flitting still is sitting ...,65.0,"And the Raven, never flitting, still is sittin...",1,85
29686,39409,137,152,Kent Brockman: ...And as my final newscast dra...,805000,True,211.0,1358.0,Kent Brockman,Air Force Base,"...And as my final newscast draws to a close, ...",and as my final newscast draws to a close im r...,51.0,"And as my final newscast draws to a close, I'm...",4,69
98822,108902,380,201,"Krusty the Clown: Gooey Martin, Willie Mimms, ...",1154000,True,139.0,3141.0,Krusty the Clown,Jazzy Goodtime's,"Gooey Martin, Willie Mimms, Drop Jaws Turner, ...",gooey martin willie mimms drop jaws turner ske...,57.0,"Gooey Martin, Willie Mimms, Drop Jaws Turner, ...",1,68
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
31434,41166,144,114,"HURLBUT: I think, Lisa, that you've been taken...",623000,True,1899.0,1403.0,HURLBUT,HISTORICAL SOCIETY,"I think, Lisa, that you've been taken in by an...",i think lisa that youve been taken in by an ob...,41.0,"I think, Lisa, that you've been taken in by an...",1,18
31350,41082,144,30,HURLBUT: (MECHANICAL COFFEE CUP VOICE) Excuse ...,197000,True,1899.0,1403.0,HURLBUT,HISTORICAL SOCIETY,"Excuse me. I didn't mean to startle you, but I...",excuse me i didnt mean to startle you but i do...,21.0,"I didn't mean to startle you, but I do love t...",2,18
31304,41037,143,217,Bart Simpson: (SO KRUSTY CAN HEAR) Krusty's ti...,1228000,True,8.0,1400.0,Bart Simpson,SPRINGFIELD MARINA - SLIP 8,"Krusty's tired of having phonies around, prete...",krustys tired of having phonies around pretend...,25.0,"Krusty's tired of having phonies around, prete...",1,18
31301,41034,143,214,"Bart Simpson: C'mon, Lis, Krusty doesn't want ...",1214000,True,8.0,1400.0,Bart Simpson,SPRINGFIELD MARINA - SLIP 8,"C'mon, Lis, Krusty doesn't want our attention ...",cmon lis krusty doesnt want our attention anym...,20.0,Let's go worship somebody who has the guts to...,2,18


In [18]:
script[script.spoken_words_split.apply(lambda x: ';' in x)].spoken_words_split.sample().values

array(['They\'re a colorful bunch; they\'ve been dubbed the "Three Musketeers'],
      dtype=object)

In [19]:
haiku_df[haiku_df.spoken_words_split.apply(lambda x: 'Pronto' in x)].iloc[1].spoken_words_split

' Promptly Package for parcel processing  Pronto Perk up, Prendergast'

In [20]:
haiku_df.n_syllables.value_counts().head(30)

[17]          1646
[9, 8]         466
[5, 12]        465
[10, 7]        428
[8, 9]         426
[12, 5]        410
[7, 10]        403
[11, 6]        376
[4, 13]        323
[6, 11]        276
[3, 14]        247
[13, 4]        220
[2, 15]        218
[15, 2]        211
[14, 3]        194
[5, 7, 5]      140
[16, 1]        138
[5, 6, 6]      119
[5, 5, 7]      111
[1, 16]        110
[7, 5, 5]      108
[4, 7, 6]       95
[5, 4, 8]       94
[4, 8, 5]       93
[8, 4, 5]       89
[6, 6, 5]       88
[4, 6, 7]       82
[4, 5, 8]       79
[7, 4, 6]       78
[10, 2, 5]      70
Name: n_syllables, dtype: int64

In [22]:
# Higher quality haikus
haiku_array = haiku_df[haiku_df.n_syllables.apply(lambda x: (x == [5, 7, 5]) | (x == [17]) | (x == [5, 12])| (x == [12, 5]))].sample().spoken_words_split.values# Search for [17], [5, 7, 5], [5, 12], [12, 5]
haiku_array

array(['All right, start over  Coarse gunpowder, ball, wadding, tamp, tamp, tamp, tamp, tamp'],
      dtype=object)

In [23]:
# Lower quality haikus
haiku_df[haiku_df.n_syllables.apply(lambda x: ((x != [5, 7, 5]) & (x != [17]) & (x != [5, 12]) & (x != [12, 5])))].sample().spoken_words_split.values# Search for [17], [5, 7, 5], [5, 12], [12, 5]

array(["That hideous man is a hero Heh  Life don't seem so hard no more"],
      dtype=object)

In [24]:
import compuglobal

simpsons = compuglobal.Frinkiac()

# Search
screencap = simpsons.search_for_screencap(haiku_array[0])

# Images/Gifs
image = screencap.get_meme_url()
gif = screencap.get_gif_url()

In [25]:
gif

'https://frinkiac.com/gif/S09E23/971820/978426.gif?b64lines=IFRIRU4sIFRFQ0hOSUNBTExZLAogU0hPVUxETidUIFlPVSBHTyBCQUNLCiBET1dOIEFORCBTVEFSVCBBTEwKIE9WRVI_IFNIVVQgVVAu'